# Integrações com o RecipeNLG - Equipe GPALT (Analistas de Cardápios)

A base de dados RecipeNLG foi a que apresentou as maiores dificuldades de integração. Os dados estão disponíveis todos em um único arquivo csv, e a grande maioria dos dados é de difícil manipulação. Por isso, foi necessário dividir o arquivo em múltiplos arquivos distintos. Isso auxilia na execução dos comandos em Python, porque as tabelas precisam ser carregadas na memória ao se utilizar essa linguagem. Certifique-se inicialmente de que o arquivo .zip foi extraído.

A divisão foi feita utilizando o comando split do Linux `dentro da pasta dos dados originais da base` após extrair o .zip, e os arquivos são divididos a cada 10 mil linhas. 
~~~bash
split -l 10000 --additional-suffix=.csv full_dataset.csv divided/
~~~

Depois da divisão, o arquivo files.txt foi criado com os nomes desses arquivos
~~~bash
ls divided/ -1 > files.txt
~~~

O header do primeiro arquivo foi apagado utilizando
~~~bash
echo "$(tail -n +2 divided/aa.csv)" > divided/aa.csv
~~~

Por fim, o header novo foi adicionado a cada arquivo um deles utilizando o comando abaixo.
~~~bash
while read p; do sed -i '1 i\id,title,ingredients,directions,link,source,NER' divided/"$p"; done <files.txt
~~~

Esses comandos não são necessários para executar o notebook da forma como os dados estão disponibilizados. Eles apenas servem como guia de como o processo foi feito.

Essa foi a única base que não foi possível integrar 100% dos ingredientes, mas devido a grande riqueza de dados ela é importante para o resultado final. O arquivo entradamenor.txt pode ser utilizado para visualizar o funcionamento dos códigos a seguir com uma entrada menor (apenas as 10 mil primeiras linhas)

### NERs

O banco de dados apresenta uma coluna chamada NERs com todos os nomes padrões dos ingredientes utilizados. Essa coluna é a coluna a partir da qual as integrações vão acontecer. No entanto, várias dessas entradas têm erros de formatação, erros de escrita e erros de não fazerem sentido (algumas entradas de ingredientes são "Children", que fazem menção a uma palavra no nome da receita e não aos ingredientes). Assim, é preciso definir um critério para a integração.

Sobre isso, foi decidido que ingredientes que aparecessem em pelo menos 10 mil receitas distintas teriam sua integração garantida. Resta, então, determinar quais são esses ingredientes.

Com a base original dividida em vários arquivos, o seguinte código Python foi utilizado para definir todos os NERs. Depois, uma consulta em SQL será realizada para definir os com maior uso.

In [1]:
import pandas as pd

# Nomes dos arquivos divididos
entrada = open("../data/raw/recipenlg/files.txt", 'r')

with open('../data/interim/recipenlg/ners.csv', 'w') as f:
    print("NER", file=f)

    for linha in entrada:
        arquivo = linha.strip()
        data = pd.read_csv('../data/raw/recipenlg/divided/%s' % arquivo)
        ners = data['NER'].values.tolist()
        receita = data['id'].values.tolist()

        for i in range(len(ners)):
            line = ners[i].lower()
            line = line.replace("[", "")
            line = line.replace("]", "")
            line = line.replace("\"", "")

            ner = line.split(", ")

            for x in ner:
                # Imprimir os NERs
                print(f'"{x}"', file=f)


Com esses dados, é possível executar em SQL um comando para filtrar essas informações. Isso será mostrado no notebook seguinte.

### Receitas

É preciso, também, separar os nomes das receitas. Isso pode ser feito em Python:

In [2]:
import pandas as pd

# Nomes dos arquivos divididos
entrada = open("../data/raw/recipenlg/files.txt", 'r')

with open('../data/interim/recipenlg/recipes.csv', 'w') as f:
    print("ID,Nome,BancoOriginal,Origem", file=f)
    for linha in entrada:
        arquivo = linha.strip()
        data = pd.read_csv('../data/raw/recipenlg/divided/%s' % arquivo)
        nome = data['title'].values.tolist()
        id = data['id'].values.tolist()

        for i in range(len(nome)):
            nome[i] = nome[i].replace("\"", "")
            print(f'"{id[i]}","{nome[i]}","RecipeNLG",', file=f)


### Ingredientes das Receitas

Agora, falta montar a relação entre ingredientes e receitas. Assim como na base do CulinaryDB, as receitas têm ingredientes associados com quantidades. No caso dessa base, a coluna "ingredients" têm esses dados, mas eles precisam ser separados de uma string única assim como na última base. Uma observação importante é que nem todo ingrediente utilizado na receita aparece na coluna NER. Nesse caso, foram observadas receitas que têm uma quantidade de ingredientes superior à quantidade de NERs (falta uma ou mais informações na coluna NER). Além disso, as receitas apresentam os ingredientes na mesma ordem dos NERs correspondentes.

Então, receitas com quantidade de NERs diferente da quantidade de ingrediente não serão associadas. Assim, as informações das quantidades podem ser extraídas a partir do seguinte código. Devido à grande quantidade de dados disponíveis, a operação pode demorar.

In [3]:
import pandas as pd
from fractions import Fraction

#Dicionarios com as palavras usadas com as unidaddes de medidas, e as outras com a conversão para unidades de medidas padrão
optional = ["to taste","own taste", "optional", "as needed", "garnish"]
unit_map = {"ounces": "ounce","ounce": "ounce","oz.": "ounce","oz": "ounce",
                "c.": "cup","c": "cup","C": "cup","cups": "cup","cup": "cup",
                "tbsp.": "tablespoon","tbsp": "tablespoon","tbs": "tablespoon","tablespoons": "tablespoon","tablespoon(s)": "tablespoon","tablespoon": "tablespoon",
                "teaspoons": "teaspoon","teaspoon": "teaspoon","tsp.": "teaspoon","tsp": "teaspoon","t.": "teaspoon",
                "lb.": "pond","lb": "pond","lbs": "pond","pound": "pond",
                "kg.": "kilogram","mg.": "miligram","grams": "gram","gram": "gram",
                "gm": "gram","g.": "gram","g": "gram",
                "qts": "quart","qts.": "quart","quarts": "quart","quart": "quart","qt.": "quart","qt": "quart",
                "gal.": "galon",
                "pints": "pint","pint": "pint","pt.": "pint",
                "liters": "liter","liter": "liter","l": "liter","ml": "mililiter",
                "inches": "inch","inch": "inch",
                "cm": "centimeter",
                "can": "can","cans": "can",
                "stick": "stick","sticks": "stick",
                "bottle": "bottle",
                "drops": "drop","drop": "drop"
               }
weight = {
        "gram": 0.001,
        "ounce" : 0.0283495,
        "pond" :  0.45359237,
        "kilogram" : 1,
    }
volumn = {
        "cup": 0.236588,
        "tablespoon" : 0.0147868,
        "teaspoon" : 0.00492892,
        "quart": 0.946353,
        "galon": 3.78541,
        "liters": 1,
        "mililiter": 0.001,
        "can": 0.3548824,
        "stick": 0.12,
        'pint': 0.473176,
    }

#Faz a primeira limpeza da quantidade, tirando pontuação, e chechando se é só um valor numérico, ela também retira dados de parenteses já que análisando os dados percebemos que quando tem parenteses com valores númericos eles representam a quantiddade e quando tem só texto ele é extensão do ingrediente, então quando tem um valor númerico substituimos a quantidade por ele
def clean(qtd):
    if isinstance(qtd, float) or isinstance(qtd, int):
        return qtd
    if qtd == '':
        return 0
    if qtd[-1]==' ':
        qtd=qtd[:-1]

    if '.' in qtd:
        qtd=qtd.replace('.','')
    if '(' in qtd and ')' in qtd:
        st=qtd.find('(')
        fm=qtd.find(')')
        if any(char.isdigit() for char in str(qtd[st+1:fm])):
            qtd=str(qtd[st+1:fm])
        else:
            qtd=qtd.replace(str(qtd[st-1:fm+1]),'')

    return qtd.lower()

#remove qualquer palavra que não tem valor numérico
def num(tex):
    a=0
    words = tex.split()
    while a < len(words):
        if not any(i.isdigit() for i in words[a]):
            words.pop(a)
        else:
            a+=1
    return ' '.join(words)

#reescreve as unidades com a escrita padrão
def padroniza(qtd):
    if isinstance(qtd, float) or isinstance(qtd, int):
        return qtd
    for a in qtd.split():
        if a in unit_map:
            un= unit_map[a]
            qtd= num(qtd) + " " + un
            return qtd
        # Se for opcional, apenas considera como 0
        elif a in optional:
            return 0
    return qtd

#faz a conversão de unidades para a unidade padrão
def conv(qtd):
    if isinstance(qtd, float) or isinstance(qtd, int):
        return qtd, 'unidade'
    qtd=qtd.split()
    valor=0
    mult=1
    medida= 'unidade'
    for val in qtd:
        if '/' in val:
            ind=True
            for char in val:
                if not(char.isdigit() or char == '/'):
                    ind = False
            if ind==True and val[-1] != '/':
                # Exemplo /2
                if (val[0] == '/'):
                    val = "1" + val
                try:
                    valor+=float(Fraction(val))
                except:
                    # Erro de processamento
                    valor = 0
        elif val.isnumeric():
            valor+=int(val)
        elif val in volumn:
            mult = 1000 * volumn[val]
            medida = 'mililitro'
        elif val in weight:
            mult = 1000 * weight[val]
            medida= 'grama'
    qtd = round(valor * mult,3)
    return qtd, medida

# Nomes dos arquivos divididos
entrada = open("../data/raw/recipenlg/files.txt", 'r')
# Visualização com menos dados
# entrada = open("../data/raw/recipenlg/entradamenor.txt", 'r')

with open('../data/interim/recipenlg/ingredientesnasreceitas.csv', 'w') as f:
    print("IDReceita,Quantidade,Unidade,Ingrediente", file=f)
    for linha in entrada:
        arquivo = linha.strip()
        dados = pd.read_csv('../data/raw/recipenlg/divided/%s' % arquivo, error_bad_lines=False, usecols=['id','title','ingredients','NER'])

        #Separando os dados em duas matrizes uma com os ingredientes com as quantidades e outra só com os ingredientes
        quantidades= dados['ingredients'].values.tolist()
        ingredientes = dados['NER'].values.tolist()
        numeros_receitas = dados['id'].values.tolist()

        #matriz com os dados
        ingre=[]

        # Verificação dos ingredientes da receita
        for a in range(len(quantidades)):
            # Quantidades: ingredientes (com unidade e quantidade)
            quantidades[a]=quantidades[a].strip('][').split(', ')
            # Ingredientes: NERs
            ingredientes[a]=ingredientes[a].strip('][').split(', ')

            # Se houver mais ingredientes do que NERs associados
            if (len(quantidades[a]) != len(ingredientes[a])):
                continue

            # Se houver algum NER que foi lido mais de uma vez (possível inconsistência)
            lista = set()
            duplicado = False

            for i in ingredientes[a]:
                if i in lista:
                    duplicado = True
                    break
                lista.add(i)

            if duplicado:
                continue

            # Processando os dados
            for b in range(len(quantidades[a])):
                # Ingrediente (com quantidade) c
                c=quantidades[a][b].strip('"')

                # Se o NER associado estiver em C, adiciono o ingrediente à receita
                if ingredientes[a][b].strip('"') in c and ingredientes[a][b].strip('"') != "":
                    # ID da receita, quantidades (sem nome do ingrediente), unidade (a ser definida), NER
                    ingre.append([numeros_receitas[a],c.replace(ingredientes[a][b].strip('"'), ""),'',ingredientes[a][b].strip('"')])
                else:
                    # A associação não é clara para o ingrediente dessa receita. Ela deve ser removida
                    while ingre != [] and ingre[-1][0] == a:
                        ingre.pop()
                    break

        #passa por todas as linhas e limpa elas
        for a in range(len(ingre)):
            ingre[a][1]=clean(ingre[a][1])
            ingre[a][1]=padroniza(ingre[a][1])
            ingre[a][1], ingre[a][2] = conv(ingre[a][1])

        #salva como banco de dados e faz o download como csv
        columns = ['recipe_id', 'qtd', 'unidade', 'ingredient']

        df = pd.DataFrame(ingre, columns=columns)
        df.to_csv(f, header=False, index=False)
